<a href="https://colab.research.google.com/github/muhammadfawad538/CNN/blob/main/Classification_through_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import zipfile
import os

zip_path = "/content/drive/MyDrive/cnn_project/archive (1).zip"  # Replace with your uploaded zip file name
extract_path = "/content/unzipped_data"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("✅ Unzipped to:", extract_path)


✅ Unzipped to: /content/unzipped_data


In [ ]:
# Update this to your actual image directory
data_dir = '/content/unzipped_data/animals/animals'  # contains 90 folders (one per animal class)


In [ ]:
import os
import shutil
import random

def split_dataset(source_dir, output_dir='dataset', train_ratio=0.8, val_ratio=0.1):
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)

    classes = os.listdir(source_dir)
    for cls in classes:
        cls_path = os.path.join(source_dir, cls)
        if not os.path.isdir(cls_path):
            continue

        images = os.listdir(cls_path)
        random.shuffle(images)

        train_count = int(train_ratio * len(images))
        val_count = int(val_ratio * len(images))
        test_count = len(images) - train_count - val_count

        splits = {
            'train': images[:train_count],
            'val': images[train_count:train_count+val_count],
            'test': images[train_count+val_count:]
        }

        for split, split_images in splits.items():
            split_dir = os.path.join(output_dir, split, cls)
            os.makedirs(split_dir, exist_ok=True)
            for img in split_images:
                src = os.path.join(cls_path, img)
                dst = os.path.join(split_dir, img)
                shutil.copy(src, dst)

# Run the split
source_dir = '/content/unzipped_data/animals/animals'
split_dataset(source_dir)


In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor()
    ]),
    'val': transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor()
    ]),
    'test': transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor()
    ]),
}

data_dir = 'dataset'  # created above
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), transform=data_transforms[x]) for x in ['train', 'val', 'test']}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=8, shuffle=True) for x in ['train', 'val', 'test']}
class_names = image_datasets['train'].classes
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"✅ Classes found: {len(class_names)}")


✅ Classes found: 90


In [ ]:
import torch.nn as nn

class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2)
        )
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 32 * 32, 128), nn.ReLU(),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.conv(x)
        x = self.fc(x)
        return x

model = SimpleCNN(num_classes=len(class_names)).to(device)


In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        model.train()
        running_loss = 0.0
        correct = 0

        for inputs, labels in dataloaders['train']:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            _, preds = torch.max(outputs, 1)
            running_loss += loss.item() * inputs.size(0)
            correct += (preds == labels).sum().item()

        epoch_loss = running_loss / len(dataloaders['train'].dataset)
        epoch_acc = correct / len(dataloaders['train'].dataset)
        print(f"Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}")

train_model(model, dataloaders, criterion, optimizer)



Epoch 1/10
Loss: 4.4945, Accuracy: 0.0171

Epoch 2/10
Loss: 4.2391, Accuracy: 0.0458

Epoch 3/10
Loss: 3.8778, Accuracy: 0.1023

Epoch 4/10
Loss: 3.3992, Accuracy: 0.1940

Epoch 5/10
Loss: 2.7449, Accuracy: 0.3419

Epoch 6/10
Loss: 2.0469, Accuracy: 0.5086

Epoch 7/10
Loss: 1.4753, Accuracy: 0.6470

Epoch 8/10
Loss: 1.0659, Accuracy: 0.7491

Epoch 9/10
Loss: 0.7517, Accuracy: 0.8197

Epoch 10/10
Loss: 0.5271, Accuracy: 0.8764
